In [1]:
# Main notebook for breakthrough infections
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "Dark2"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
# plt.rcParams["image.cmap"] = "tab10"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
    'axes.labelsize': fs_label,
    'axes.titlesize': fs_label+4,
    'xtick.labelsize': fs_label,
    'ytick.labelsize': fs_label, 
    'legend.fontsize': fs_label, 
    'lines.markersize': 10,
    'lines.linewidth': 3
}
plt.rcParams.update(parameters)
%matplotlib widget  
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

R0 = 4.7

S = lambda V, Preval : (1-V)*(1-Preval)
Re = lambda R0, V, Preval: R0*S(V, Preval)
V = lambda Re, R0, Preval: 1 - (Re/R0)*(1/(1-Preval))
P = lambda Re, R0, Vacc: 1 - (Re/R0)*(1/(1-Vacc))

saveFigures is set to: True
Done loading packages


In [2]:
data_raw_df = pd.read_csv("owid-covid-data.csv")
# data_raw_df.columns

In [3]:
all_loc = data_raw_df.location.unique()

locs = ["Denmark", "Latvia", "Poland", "Estonia", "Lithuania", "Spain","Romania","Georgia"]

loc_filt = data_raw_df.location.isin(locs)

data_df = data_raw_df.loc[loc_filt]

#fig1, ax1 = plt.subplots()
#fig2, ax2 = plt.subplots()
fig, ax = plt.subplots(figsize=(12,12))

scatterstyle = {
    # "color":'tab:red', 
    # "linestyle":':', 
    # "linewidth":5, 
    "marker":'o', 
    # "markersize":5, 
    # "edgecolors":'tab:red'
    # alpha":0.6
}

scatter_df = pd.DataFrame(columns=["loc", "vacc", "preval"])

resolution = 1001

xlab = "Vaccine coverage"
ylab = "Prevalence (Infections)"


# Adjust here, for the visible range in plot:
X = vacc_list = np.linspace(0, 1, resolution)
Y = preval_list = np.linspace(0, 1, resolution)
Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc


r0 = 4.7
# res = np.linspace(0.05,0.95,numAreas)*r0
res = np.arange(0,r0,0.25)
numAreas = len(res)

#ax.pcolor(X, Y, Z)
# ax.contourf(X, Y, Z, 100, cmap="viridis")
ax.contourf(X, Y, Z, numAreas, cmap="Greys")

for i, re in enumerate([1]):
    IsoVac = np.linspace(0,1-(re/r0),100)
    ax.plot(IsoVac, (lambda V: P(re, r0, V))(IsoVac), alpha=0.8, color="orange", label=int(i==0)*"$R_{e} = 1$-Isocline")

heard_immune_color = lambda r0: int(r0<1)*"green" + int(r0>1)*"red" + int(r0==1)*"orange"
hi_colors = [heard_immune_color(r) for r in res]

# CS = ax.contour(X, Y, Z, levels=res, colors=hi_colors) # , cmap="plasma") #, label="$R_{0}$-isoclines")
CS = ax.contour(X, Y, Z, levels=res, colors=hi_colors) # , cmap="plasma") #, label="$R_{0}$-isoclines")
clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/r0]
#ax.scatter(*list(zip(*clab_pos)))
ax.clabel(CS, inline=True, fontsize=10, manual=clab_pos)

for loc, loc_data in data_df.groupby(["location"]):
    
    Prevals = loc_data.total_cases_per_million
    Preval = Prevals.dropna().values[-1]/int(1e6)

    Vaccs = loc_data.people_fully_vaccinated_per_hundred
    Vacc = Vaccs.dropna().values[-1]/100
    
    # Comment this line to not show history...
    ax.plot(Vaccs/100, Prevals/int(1e6), alpha=0.3)
    
    # present state in this parameter space:
    ax.scatter(Vacc, Preval, **scatterstyle)
    ax.annotate(loc, (Vacc, Preval))

    # Currently not used:
    row = {"loc":loc, "vacc":Vacc, "preval":Preval}
    scatter_df = scatter_df.append(row, ignore_index=True)
    print(loc, Vacc, Preval)
    
    #ax1.plot(loc_data.date, Prevals, label=loc)
    #ax2.plot(loc_data.date, Vaccs, label=loc)
    
ax.legend()
#ax2.legend()
#data_df
# ax.set_ylim([0,0.3])
# ax.scatter(scatter_df.vacc, scatter_df.preval, **scatterstyle)
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)

# plt.savefig("figs/parameterspace_vacc_prevalence.png")
# plt.savefig("figs/parameterspace_vacc_prevalence.pdf")
scatter_df

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Denmark 0.7565000000000001 0.064025058
Estonia 0.5511 0.131631135
Georgia 0.2279 0.168490012
Latvia 0.5068 0.101086594
Lithuania 0.6118 0.138514541
Poland 0.5227 0.077917718
Romania 0.2974 0.07770188800000001
Spain 0.7934 0.106765311


,loc,vacc,preval
0,Denmark,0.7565,0.064025
1,Estonia,0.5511,0.131631
2,Georgia,0.2279,0.168490
3,Latvia,0.5068,0.101087
4,Lithuania,0.6118,0.138515
5,Poland,0.5227,0.077918
6,Romania,0.2974,0.077702
7,Spain,0.7934,0.106765


In [4]:

from matplotlib.colors import LinearSegmentedColormap
# cmap = LinearSegmentedColormap.from_list('unnamed', 
#                                                     [
#                                                         (0,1,0),
#                                                         (1,0.9,0.7),
#                                                         (1,0,0),
#                                                         (1,0,0),
#                                                         (1,0,0),
#                                                         (1,0,0),
#                                                     ] 
#                                                     , N=numAreas)

In [48]:

# currentDate = np.datetime64('2021-08')

counter = 0
for currentDate in np.arange(np.datetime64('2021-01'),np.datetime64('2021-12')):

    all_loc = data_raw_df.location.unique()

    locs = ["Denmark", "Latvia", "Poland", "Estonia", "Lithuania", "Spain","Romania","Georgia","Germany"]

    loc_filt = data_raw_df.location.isin(locs)

    data_df = data_raw_df.loc[loc_filt]
    data_df['date'] = pd.to_datetime(data_df.date)

    toUse = (data_df.date <= currentDate)
    data_df_period = data_df[toUse]
    # toUse

    #fig1, ax1 = plt.subplots()
    #fig2, ax2 = plt.subplots()
    fig, ax = plt.subplots(figsize=(12,12))

    scatterstyle = {
        # "color":'tab:red', 
        # "linestyle":':', 
        # "linewidth":5, 
        "marker":'o', 
        # "markersize":5, 
        # "edgecolors":'tab:red'
        # alpha":0.6
    }

    scatter_df = pd.DataFrame(columns=["loc", "vacc", "preval"])

    resolution = 1001

    xlab = "Vaccine coverage"
    ylab = "Prevalence (Infections)"



    minX = 0
    maxX = 1
    minY = 0
    maxY = 1

    # minX = 0.3
    # maxX = 1
    # minY = 0
    # maxY = 0.5

    # Adjust here, for the visible range in plot:
    X = vacc_list = np.linspace(minX, maxX, resolution)
    Y = preval_list = np.linspace(minY,maxY, resolution)
    Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc



    r0 = 4.7
    # res = np.linspace(0.05,0.95,numAreas)*r0
    res = np.arange(0,5,0.25)
    numAreas = len(res)

    cmap = LinearSegmentedColormap.from_list('unnamed', 
                                                        [
                                                            (0,1,0),
                                                            (1,0.9,0.7),
                                                            (1,0,0),
                                                            (1,0,0),
                                                            (1,0,0),
                                                            (1,0,0),
                                                        ] 
                                                        , N=numAreas)

    #ax.pcolor(X, Y, Z)
    # ax.contourf(X, Y, Z, 100, cmap="viridis")
    # ax.contourf(X, Y, Z, numAreas, cmap=cmap)
    ax.contourf(X, Y, Z, levels=res,cmap=cmap)


    for i, re in enumerate([1]):
        IsoVac = np.linspace(0,1-(re/r0),100)
        ax.plot(IsoVac, (lambda V: P(re, r0, V))(IsoVac), linewidth = 10,alpha=0.5,zorder=5, color="grey", label=int(i==0)*"$R_{e} = 1$-Isocline")

    # heard_immune_color = lambda r0: int(r0<1)*"green" + int(r0>1)*"red" + int(r0==1)*"orange"
    # hi_colors = [heard_immune_color(r) for r in res]

    # CS = ax.contour(X, Y, Z, levels=res, colors=hi_colors) # , cmap="plasma") #, label="$R_{0}$-isoclines")


    for loc, loc_data in data_df_period.groupby(["location"]):
        
        # Prevals = loc_data.total_cases_per_million
        # Preval = Prevals.dropna().values[-1]/int(1e6)
        Prevals = 2* loc_data.total_cases_per_million
        Preval =  Prevals.dropna().values[-1]/int(1e6)

        Vaccs = loc_data.people_fully_vaccinated_per_hundred
        try:
            Vacc = Vaccs.dropna().values[-1]/100
        except:
            Vacc = 0
        
        # Comment this line to not show history...
        ax.plot(Vaccs/100, Prevals/int(1e6),'.', alpha=0.3,markersize=4)
        
        # present state in this parameter space:
        ax.scatter(Vacc, Preval, **scatterstyle,zorder=12)
        # curOffset = 0.01
        # ax.annotate(loc, (Vacc+curOffset, Preval+curOffset),fontsize=16,zorder=11)
        ax.annotate(loc, xy=(Vacc, Preval), xytext=(12, 5),textcoords='offset pixels',fontsize=16,zorder=11,bbox=dict(boxstyle="round4,pad=0.2", fc="w",alpha=0.7, ec="k", lw=2))
        # ax.annotate(loc, xy=(Vacc, Preval), xytext=(12, 5),textcoords='offset pixels',fontsize=16,zorder=12)
        # ax.text(Vacc, Preval,loc ,fontsize=16,zorder=11)

        # Currently not used:
        row = {"loc":loc, "vacc":Vacc, "preval":Preval}
        scatter_df = scatter_df.append(row, ignore_index=True)
        
    CS = ax.contour(X, Y, Z, levels=res, zorder=6,colors='k') 
    # clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/r0]
    # clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/r0]
    # #ax.scatter(*list(zip(*clab_pos)))
    # ax.clabel(CS, inline=True, fontsize=10, manual=clab_pos)
    # clab_pos = [(y,0.025) for y in res/r0]
    # clab_pos = [(y,0.025) for y in np.linspace(0,1,numAreas)]
    numLabelsToShow = numAreas-6
    clab_pos = list(zip(np.linspace(minX,maxX,numLabelsToShow),np.linspace(minY,maxY,numLabelsToShow)))
    ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10,manual=clab_pos)
    # ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10)
        
    # ax.axis('equal')
    ax.set_xlim([minX,maxX])
    ax.set_ylim([minY,maxY])

    # ax.legend()
    #ax2.legend()
    #data_df
    # ax.set_ylim([0,0.3])
    # ax.scatter(scatter_df.vacc, scatter_df.preval, **scatterstyle)
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)
    # plt.tight_layout()
    plt.show()
    
    # plt.savefig(f'figs/anim/ParameterSpace_{currentDate}')
    plt.savefig(f'figs/anim/frame{counter:1.0f}')
    counter = counter+1

<ipython-input-48-bd68ddd77baf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['date'] = pd.to_datetime(data_df.date)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-48-bd68ddd77baf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['date'] = pd.to_datetime(data_df.date)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-48-bd68ddd77baf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['date'] = pd.to_datetime(data_df.date)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-48-bd68ddd77baf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['date'] = pd.to_datetime(data_df.date)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
# data_df['date'] = pd.to_datetime(data_df.date)

# currentDate = np.datetime64('2021-08-01')
# toUse = data_df.date <= currentDate
# toUse

# Vaccs.dropna().values[-1]


IndexError: index -1 is out of bounds for axis 0 with size 0